In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
from pathlib import Path
import re
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
from pathlib import Path
import shutil
from pathlib import Path
# from pyphocorehelpers.plotting.media_output_helpers import vertical_image_stack, horizontal_image_stack, image_grid # used in `_subfn_build_combined_output_images`
from pyphocorehelpers.image_helpers import ImageHelpers
from copy import deepcopy
import numpy as np
from pyjsoncanvas import (
    Canvas,
    TextNode,
    FileNode,
    LinkNode,
    GroupNode,
    GroupNodeBackgroundStyle,
    Edge,
    Color,
)

from pyphocorehelpers.Filesystem.obsidian_canvas_helpers import ObsidianCanvasHelper


# Multi sessions

In [ ]:
## Home/Apogee:
# sessions_export_folder = Path('K:/scratch/collected_outputs/2025-05-22')
# canvas_folders_url = Path(r"D:/PhoGlobalObsidian2022/🌐🧠 Working Memory/Pho-Kamran Paper 2024/_programmatic_test")


## Lab/Linux:
sessions_export_folder = Path('/home/halechr/FastData/collected_outputs/2025-05-27').resolve()
canvas_folders_url = Path(r"/home/halechr/cloud/Cryptomator/PhoGlobalObsidian2022/🌐🧠 Working Memory/Pho-Kamran Paper 2024/_programmatic_test")


is_single_canvas = True
# is_single_canvas = False
added_groups_dict, write_modified_canvas_path = ObsidianCanvasHelper.build_canvas_for_exported_session_posteriors(sessions_export_folder = sessions_export_folder,
                                                    intra_session_v_spacing = 3000, intra_laps_and_pbes_v_spacing = 1000, is_single_canvas = is_single_canvas, common_add_images_to_canvas_kwargs = dict(debug_print=False, canvas_image_node_scale=0.1), image_glob="p_x_given_n*.png",
                                                    canvas_folders_url=canvas_folders_url,
                                                )

## Existing Canvas Node Manipulation

In [ ]:
existing_canvas_path: Path = Path('/home/halechr/cloud/Cryptomator/PhoGlobalObsidian2022/🌐🧠 Working Memory/Pho-Kamran Paper 2024/_programmatic_test/ALL_SESSIONS.canvas').resolve()
assert existing_canvas_path.exists()
existing_canvas_path

# Pre-Classmethod Notebook version of `ObsidianCanvasHelper.build_canvas_for_exported_session_posteriors`

In [ ]:
from pathlib import Path


intra_session_v_spacing: int = 3000
intra_laps_and_pbes_v_spacing: int = 1000
# is_single_canvas: bool = True
is_single_canvas: bool = False
common_add_images_to_canvas_kwargs = dict(debug_print=False)

## INPUTS: intra_session_v_spacing, intra_laps_and_pbes_v_spacing, is_single_canvas

added_groups_dict = {}

# image_glob="merged_*.png"
image_glob="p_x_given_n*.png"

canvas_folders_url = Path(r"D:/PhoGlobalObsidian2022/🌐🧠 Working Memory/Pho-Kamran Paper 2024/_programmatic_test") ## 
## INPUTS: canvas_folders_url

# Path to the directory you want to iterate through
base_path = Path('K:/scratch/collected_outputs/2025-05-21')



# Create a new canvas
if is_single_canvas:
    print(f'creating new Canvas as none was provided!')
    write_mode = 'x' # set write mode to create only so it doesn't overwrite an existing cnvas
    target_canvas = Canvas(nodes=[], edges=[])
    initial_x = 0
    initial_y = 0    

else:
    target_canvas = None
    initial_x = 0
    initial_y = 0
    

# Iterate through only the directories (folders)
for session_folder in [item for item in base_path.iterdir() if item.is_dir()]:
    print(f"Found folder: {session_folder.name}")
    session_name: str = session_folder.name.removesuffix('_weighted_position_posterior')
    print(f'\tsession_name: {session_name}')
    

    if not is_single_canvas:
        # Do something with each folder
        write_modified_canvas_path: Path = canvas_folders_url.joinpath(f'_programmatic_test_{session_name}.canvas')
        # image_folder_path: Path = Path(r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2025-05-21/gor01_two_2006-6-07_16-40-19_normal_computed_[1, 2]_5.0/ripple/psuedo2D_nan_filled/raw_rgba').resolve()
        target_canvas = None ## start with a blank canvas each time
        
    else:
        ## single_canvas mode: set no save URL so it doesn't write out to file
        write_modified_canvas_path = None

    image_folder_path: Path = session_folder.joinpath('laps/psuedo2D_nan_filled/raw_rgba').resolve()
    target_canvas, laps_group_node, _write_status = ObsidianCanvasHelper.add_images_to_canvas(image_folder_path=image_folder_path, image_glob=image_glob, target_canvas=target_canvas, write_modified_canvas_path=write_modified_canvas_path, override_write_mode='w', image_group_name=f'Laps - {session_name}', initial_x = initial_x, initial_y = initial_y, **common_add_images_to_canvas_kwargs)
    
    ## use the existing canvas
    initial_y = initial_y + intra_laps_and_pbes_v_spacing
    image_folder_path: Path = session_folder.joinpath('ripple/psuedo2D_nan_filled/raw_rgba').resolve()
    target_canvas, pbes_group_node, _write_status = ObsidianCanvasHelper.add_images_to_canvas(image_folder_path=image_folder_path, image_glob=image_glob, target_canvas=target_canvas, write_modified_canvas_path=write_modified_canvas_path, override_write_mode='w', image_group_name=f'PBEs - {session_name}', initial_x = initial_x, initial_y=initial_y, **common_add_images_to_canvas_kwargs)
    
    ## Build session group
    group_node: GroupNode = ObsidianCanvasHelper.group_wrapping_nodes(children_nodes=(laps_group_node, pbes_group_node), group_name=f'{session_name}', group_padding=(100, 90))
    target_canvas.add_node(group_node)
    # added_groups_dict[session_name] = group_node
    added_groups_dict[f'{session_name}'] = {'session': group_node, 'laps': laps_group_node, 'pbes': pbes_group_node}
    
    if is_single_canvas:
        initial_x = 0
        initial_y = initial_y + intra_session_v_spacing ## spacing of 5000 between sessions

    else:
        ## reset canvas so new one is created
        target_canvas = None
        ## re-zero
        initial_x = 0
        initial_y = 0    

    # target_canvas, _write_status


if is_single_canvas:
    ## do final save
    write_modified_canvas_path: Path = canvas_folders_url.joinpath(f'ALL_SESSIONS.canvas') ## session-specific canvas save URL
    _write_status = ObsidianCanvasHelper.save(canvas=target_canvas, canvas_url=write_modified_canvas_path, write_mode='w')
    print(f'Finished writing to "{write_modified_canvas_path.as_posix()}"')
## OUTPUTS: added_groups_dict, write_modified_canvas_path




In [ ]:
329
389
380


In [ ]:
if is_single_canvas:
    ## do final save
    write_modified_canvas_path: Path = canvas_folders_url.joinpath(f'ALL_SESSIONS.canvas') ## session-specific canvas save URL
    _write_status = ObsidianCanvasHelper.save(canvas=target_canvas, canvas_url=write_modified_canvas_path, write_mode='w')

In [ ]:


laps_group_node.x, laps_group_node.y, laps_group_node.width, laps_group_node.height



In [ ]:
pbes_group_node

In [ ]:
# target_canvas.get_node()
target_canvas.__dir__()


In [ ]:
target_canvas.nodes

# Single Session Test of `ObsidianCanvasHelper.add_images_to_canvas`


In [ ]:
canvas_url = Path(r"D:/PhoGlobalObsidian2022/🌐🧠 Working Memory/Pho-Kamran Paper 2024/2025-05-15 - Pho Sorted Events.canvas")
write_modified_canvas_path: Path = canvas_url.with_name(f'_programmatic_test.canvas')
loaded_canvas = ObsidianCanvasHelper.load(canvas_url=canvas_url)

# image_folder_path: Path = Path(r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2025-05-21/gor01_two_2006-6-07_16-40-19_normal_computed_[1, 2]_5.0/ripple/psuedo2D_nan_filled/raw_rgba').resolve()
image_folder_path: Path = Path(r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2025-05-21/gor01_two_2006-6-07_16-40-19_normal_computed_[1, 2]_5.0/laps/psuedo2D_nan_filled/raw_rgba').resolve()

target_canvas, _write_status = ObsidianCanvasHelper.add_images_to_canvas(image_folder_path=image_folder_path, image_glob="p_x_given_n*.png", target_canvas=None, write_modified_canvas_path=write_modified_canvas_path, override_write_mode='w', image_group_name='test')
# target_canvas, _write_status = ObsidianCanvasHelper.add_images_to_canvas(image_folder_path=image_folder_path, image_glob="merged_*.png", target_canvas=None, write_modified_canvas_path=write_modified_canvas_path, override_write_mode='w')
# target_canvas, _write_status



In [ ]:
canvas_url = Path(r"D:/PhoGlobalObsidian2022/🌐🧠 Working Memory/Pho-Kamran Paper 2024/2025-05-15 - Pho Sorted Events.canvas")
canvas_url


In [ ]:

## save the canvas back to a file
test_modified_canvas_url: Path = canvas_url.with_name(f'_programmatic_test.canvas')
test_modified_canvas_url

## INPUTS: active_canvas
# Save the canvas as JSON
json_str = active_canvas.to_json()
# Load JSON from a string
with open(test_modified_canvas_url, 'w', encoding='utf-8') as f:
    # canvas_json = json.load(f)
	_status_code = f.write(json_str)
	if _status_code:
		print(f'writing error code: {_status_code}')

# Testing from scratch

In [ ]:

from pyjsoncanvas import (
    Canvas,
    TextNode,
    FileNode,
    LinkNode,
    GroupNode,
    GroupNodeBackgroundStyle,
    Edge,
    Color,
)

# Create a new canvas
canvas = Canvas(nodes=[], edges=[])

# Add some nodes
text_node = TextNode(x=100, y=100, width=200, height=100, text="Hello, world!")
canvas.add_node(text_node)

file_node = FileNode(x=300, y=100, width=100, height=100, file="/path/to/file.png")
canvas.add_node(file_node)

# Add an edge
edge = Edge(
    fromNode=text_node.id,
    fromSide="bottom",
    toNode=file_node.id,
    toSide="top",
    color=Color("#FF0000"),
    label="Edge 1",
)
canvas.add_edge(edge)

# Save the canvas as JSON
json_str = canvas.to_json()

# Load the canvas from JSON
loaded_canvas = Canvas.from_json(json_str)

# Get a node
node = loaded_canvas.get_node(text_node.id)

# Get connections for a node
connections = loaded_canvas.get_connections(text_node.id)